## Imports

In [1]:
import os
import requests
from IPython.display import display, HTML
from tabulate import tabulate
import urllib.request


## Helper funcs. 

In [2]:

def fetch_all_data(url):
    """Fetch all data from the given URL."""
    print("retrieving data from", url)
    counter = 0 
    documents = []
    while url and counter < 1000:
        response = requests.get(url)
        data = response.json()
        documents.extend(data.get('value', []))
        url = data.get('@odata.nextLink', None)  # Get the next link if available
        counter += 1
    return documents

def truncate_string(s, max_length):
    """Truncate string to the max_length with ellipsis if needed."""
    return s if len(s) <= max_length else s[:max_length - 3] + '...'

def construct_url(base_url, filters=None, select=None, expand=None, skip=None, top=None, orderby=None, count=False):
    """Function to construct URL with OData filters"""
    query_params = []

    if filters:
        query_params.append(f"$filter={filters}")
    if select:
        query_params.append(f"$select={select}")
    if expand:
        query_params.append(f"$expand={expand}")
    if skip:
        query_params.append(f"$skip={skip}")
    if top:
        query_params.append(f"$top={top}")
    if orderby:
        query_params.append(f"$orderby={orderby}")
    if count:
        query_params.append(f"$count=true")

    if query_params:
        return f"{base_url}?" + "&".join(query_params)
    else:
        return base_url

## Letsgoo

In [3]:
base_url = 'https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/'
documentation_url = "https://opendata.tweedekamer.nl/documentatie/"

# Filters en params
filters = "Kamer eq 'tweede kamer' and year(datum) eq 2024"
select = None # Kan bijvoorbeeld zijn: 'id, naam, partij'
expand = "Verslag" # Bedoeld om child-data op te vragen, 
skip = None # Skip aantal records
top = None # Top aantal records
orderby = "Datum desc" # Orderby asc, desc
count = False # Count aantal records

# Maak url
combined_url = construct_url(base_url + "Vergadering", filters, select, expand, skip, top, orderby, count)
print(combined_url)

https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/Vergadering?$filter=Kamer eq 'tweede kamer' and year(datum) eq 2024&$expand=Verslag&$orderby=Datum desc


In [4]:
vergadering_data = fetch_all_data(combined_url)

amount_of_rows = 10
data = []

for vergadering in vergadering_data:
    titel = truncate_string(vergadering['Titel'], 50)
    data.append([titel, vergadering['Datum'], vergadering['Zaal'], vergadering.get('Verslag')[-1].get('Soort')])

print(tabulate(data[:amount_of_rows], headers=['Titel', 'Datum', 'Zaal', 'Verslag soort']))

retrieving data from https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/Vergadering?$filter=Kamer eq 'tweede kamer' and year(datum) eq 2024&$expand=Verslag&$orderby=Datum desc
Titel                                    Datum                      Zaal                      Verslag soort
---------------------------------------  -------------------------  ------------------------  ----------------
92e vergadering, donderdag 4 juli 2024   2024-07-04T00:00:00+02:00  Plenaire zaal             Tussenpublicatie
Mestbeleid                               2024-07-04T00:00:00+02:00  Thorbeckezaal             Tussenpublicatie
91e vergadering, woensdag 3 juli 2024    2024-07-03T00:00:00+02:00  Plenaire zaal             Tussenpublicatie
NAVO                                     2024-07-03T00:00:00+02:00  Thorbeckezaal             Tussenpublicatie
90e vergadering, donderdag 27 juni 2024  2024-06-27T00:00:00+02:00  Plenaire zaal             Tussenpublicatie
vmbo                                     2024-06

### Download verslagen

In [5]:
# We'll just be downloading 5 for now
for vergadering in vergadering_data[:5]:
    # Get (latest) verslag from vergadering
    verslag_id = vergadering['Verslag'][-1]['Id']
    verslag_url = f"{base_url}Verslag/{verslag_id}/resource"
    # Download the verslag URL in XML format
    urllib.request.urlretrieve(verslag_url, f'verslag-data/{verslag_id}.xml')

## Parse verslag XML to gain some insights

In [6]:
import os
import xml.etree.ElementTree as ET

# Get the path of the first XML file in the directory
xml_dir = './verslag-data/'
xml_files = os.listdir(xml_dir)
xml_file_path = os.path.join(xml_dir, xml_files[0])

print(f"Analyzing first XML file: {xml_file_path}")

# Parse the XML file
tree = ET.parse(xml_file_path)
root = tree.getroot()

# Print the root tag to verify the file structure
print(f"Root tag: {root.tag}")

Analyzing first XML file: ./verslag-data/0a69c26d-1b24-4012-b3c2-f827c0e65634.xml
Root tag: {http://www.tweedekamer.nl/ggm/vergaderverslag/v1.0}vlosCoreDocument


In [7]:
# Print all the tags in the XML file to understand its structure
for elem in tree.iter():
    print(elem.tag)

{http://www.tweedekamer.nl/ggm/vergaderverslag/v1.0}vlosCoreDocument
{http://www.tweedekamer.nl/ggm/vergaderverslag/v1.0}vergadering
{http://www.tweedekamer.nl/ggm/vergaderverslag/v1.0}titel
{http://www.tweedekamer.nl/ggm/vergaderverslag/v1.0}zaal
{http://www.tweedekamer.nl/ggm/vergaderverslag/v1.0}vergaderjaar
{http://www.tweedekamer.nl/ggm/vergaderverslag/v1.0}vergaderingnummer
{http://www.tweedekamer.nl/ggm/vergaderverslag/v1.0}datum
{http://www.tweedekamer.nl/ggm/vergaderverslag/v1.0}aanvangstijd
{http://www.tweedekamer.nl/ggm/vergaderverslag/v1.0}sluiting
{http://www.tweedekamer.nl/ggm/vergaderverslag/v1.0}activiteit
{http://www.tweedekamer.nl/ggm/vergaderverslag/v1.0}titel
{http://www.tweedekamer.nl/ggm/vergaderverslag/v1.0}onderwerp
{http://www.tweedekamer.nl/ggm/vergaderverslag/v1.0}parlisid
{http://www.tweedekamer.nl/ggm/vergaderverslag/v1.0}aanvangstijd
{http://www.tweedekamer.nl/ggm/vergaderverslag/v1.0}eindtijd
{http://www.tweedekamer.nl/ggm/vergaderverslag/v1.0}voortzettin

In [8]:
# Define namespaces if your XML uses them
namespace = {'ns': 'http://www.tweedekamer.nl/ggm/vergaderverslag/v1.0'}


In [9]:
# Retrieve all <spreker> elements and their respective children
# Adjust the XPath query if namespaces are used
spreker_elements = root.findall('.//ns:spreker', namespace)

In [15]:
tag = '{http://www.tweedekamer.nl/ggm/vergaderverslag/v1.0}'


class Spreker:
    def __init__(self, fractie, aanhef, verslagnaam, weergavenaam, voornaam, achternaam, functie, tekst=''):
        self.fractie = fractie
        self.aanhef = aanhef
        self.verslagnaam = verslagnaam
        self.weergavenaam = weergavenaam
        self.voornaam = voornaam
        self.achternaam = achternaam
        self.functie = functie
        self.tekst = tekst
        
        
    def __repr__(self):
        return (f"Spreker(fractie='{self.fractie}', aanhef='{self.aanhef}', verslagnaam='{self.verslagnaam}', "
                f"weergavenaam='{self.weergavenaam}', voornaam='{self.voornaam}', achternaam='{self.achternaam}', "
                f"functie='{self.functie}', tekst='{self.tekst}')")

    def __eq__(self, other):
        if isinstance(other, Spreker):
            return (self.fractie, self.voornaam, self.achternaam) == (other.fractie, other.voornaam, other.achternaam)
        return False

    def __hash__(self):
        return hash((self.fractie, self.voornaam, self.achternaam))

    
def from_xml_element(element):
    return Spreker(
        element.find(f'{tag}fractie').text if element.find(f'{tag}fractie') is not None else '',
        element.find(f'{tag}aanhef').text if element.find(f'{tag}aanhef') is not None else '',
        element.find(f'{tag}verslagnaam').text if element.find(f'{tag}verslagnaam') is not None else '',
        element.find(f'{tag}weergavenaam').text if element.find(f'{tag}weergavenaam') is not None else '',
        element.find(f'{tag}voornaam').text if element.find(f'{tag}voornaam') is not None else '',
        element.find(f'{tag}achternaam').text if element.find(f'{tag}achternaam') is not None else '',
        element.find(f'{tag}functie').text if element.find(f'{tag}functie') is not None else ''
    )

In [50]:
from tabulate import tabulate

sprekers_set = set()
sprekers = []

for spreker_element in spreker_elements:
    spreker = from_xml_element(spreker_element)
    if (spreker.fractie, spreker.verslagnaam, spreker.weergavenaam) not in sprekers_set:
        sprekers_set.add((spreker.fractie, spreker.verslagnaam, spreker.weergavenaam))
        sprekers.append(spreker)

table = [[spreker.fractie, spreker.aanhef, spreker.verslagnaam, spreker.weergavenaam, spreker.voornaam, spreker.achternaam, spreker.functie] for spreker in sprekers]
headers = ['Fractie', 'Aanhef', 'Verslagnaam', 'Weergavenaam', 'Voornaam', 'Achternaam', 'Functie']

print(tabulate(table, headers=headers))

Fractie          Aanhef    Verslagnaam        Weergavenaam         Voornaam    Achternaam    Functie
---------------  --------  -----------------  -------------------  ----------  ------------  --------------------------------------------------------
VVD              Mevrouw   Tielen             Tielen               Judith      Tielen        lid Tweede Kamer
NSC              Mevrouw   Postma             Postma               Wytske      Postma        lid Tweede Kamer
GroenLinks-PvdA  De heer   White              White                Raoul       White         lid Tweede Kamer
D66              De heer   Tjeerd de Groot    Groot de (Tjeerd)    Tjeerd      Groot de      lid Tweede Kamer
ChristenUnie     De heer   Grinwis            Grinwis              Pieter      Grinwis       lid Tweede Kamer
SGP              De heer   Flach              Flach                André       Flach         lid Tweede Kamer
CDA              Mevrouw   Inge van Dijk      Dijk van (Inge)      Inge        Dijk van  

In [86]:
tag = '{http://www.tweedekamer.nl/ggm/vergaderverslag/v1.0}'

def parse_vergadering(element):
    data = {}
    for idx, child in enumerate(element):
        if child.tag != f"{tag}activiteit":
            data[child.tag.replace(tag, '')] = child.text
        else:
            data[f"activiteit{idx}"] = parse_activiteit(child)
    return data

def parse_activiteit(element):
    activiteit_data = {
        "soort": element.get('soort'),
        "objectid": element.get('objectid')
    }
    for child in element:
        activiteit_data[child.tag.replace(tag, '')] = child.text
    activiteit_data["sprekers"] = find_sprekers(element)
    return activiteit_data

def find_sprekers(element):
    sprekers = set()
    for child in element:
        if child.tag == f"{tag}spreker":
            spreker = from_xml_element(child)
            sprekers.add(spreker)
        else:
            sprekers.update(find_sprekers(child))
    return list(sprekers)

vergadering_data = {}
for element in root:
    if element.tag == f"{tag}vergadering":
        vergadering_data = parse_vergadering(element)

vergadering_data

{'titel': '45e vergadering, woensdag 28 februari 2024',
 'zaal': 'Plenaire zaal',
 'vergaderjaar': '2023-2024',
 'vergaderingnummer': '45',
 'datum': '2024-02-28T00:00:00',
 'aanvangstijd': '2024-02-28T10:15:00',
 'sluiting': '2024-02-28T00:00:00',
 'activiteit7': {'soort': 'Opening',
  'objectid': '53fde46e-452d-44a1-af8b-0d1165e8c82b',
  'titel': 'Opening',
  'onderwerp': 'Opening',
  'parlisid': None,
  'aanvangstijd': '2024-02-28T10:15:31',
  'eindtijd': '2024-02-28T10:15:43',
  'voortzetting': 'false',
  'activiteithoofd': '\n                ',
  'sprekers': [Spreker(fractie='VVD', aanhef='Mevrouw', verslagnaam='Tielen', weergavenaam='Tielen', voornaam='Judith', achternaam='Tielen', functie='lid Tweede Kamer', tekst='')]},
 'activiteit8': {'soort': 'Mededelingen',
  'objectid': '21f52447-f22c-4ad4-ba8e-e47077eb29dd',
  'titel': 'Mededelingen',
  'onderwerp': 'Mededelingen',
  'parlisid': None,
  'aanvangstijd': '2024-02-28T10:15:43',
  'eindtijd': '2024-02-28T10:16:01',
  'voortze

In [136]:
activiteit_id = None
current_spreker = None

for vergadering_element in root.iter():
    if vergadering_element.tag == f"{tag}activiteit":
        activiteit_id = vergadering_element.get('objectid')
    if vergadering_element.tag == f"{tag}spreker":
        current_spreker = from_xml_element(vergadering_element)
    if vergadering_element.tag == f"{tag}tekst":
        def print_all_text(element):
            if current_spreker:
                for key in vergadering_data.keys():
                    if type(vergadering_data.get(key)) == dict:
                        if vergadering_data.get(key).get('objectid') == activiteit_id:
                            for idx, activiteit_spreker in enumerate(vergadering_data.get(key).get('sprekers')):
                                if current_spreker == activiteit_spreker:
                                    vergadering_data.get(key).get('sprekers')[idx].tekst += element.text
                        
                current_spreker.tekst += element.text
            for child in element:
                print_all_text(child)
        print_all_text(vergadering_element)
        
vergadering_data

{'titel': '45e vergadering, woensdag 28 februari 2024',
 'zaal': 'Plenaire zaal',
 'vergaderjaar': '2023-2024',
 'vergaderingnummer': '45',
 'datum': '2024-02-28T00:00:00',
 'aanvangstijd': '2024-02-28T10:15:00',
 'sluiting': '2024-02-28T00:00:00',
 'activiteit7': {'soort': 'Opening',
  'objectid': '53fde46e-452d-44a1-af8b-0d1165e8c82b',
  'titel': 'Opening',
  'onderwerp': 'Opening',
  'parlisid': None,
  'aanvangstijd': '2024-02-28T10:15:31',
  'eindtijd': '2024-02-28T10:15:43',
  'voortzetting': 'false',
  'activiteithoofd': '\n                ',
  'sprekers': [Spreker(fractie='VVD', aanhef='Mevrouw', verslagnaam='Tielen', weergavenaam='Tielen', voornaam='Judith', achternaam='Tielen', functie='lid Tweede Kamer', tekst='
                                   
                                       De voorzitterGoedemorgen allemaal. Ik open de vergadering van
                                           woensdag 28 februari 2024.
                                   
                        